In [1]:
# ЗАДАНИЕ 2.1

(1.1 * 5.1 + 2.3 * 6.2 + 5.1 * 1.1) / ((1.2**2 + 2.3**2 + 5.1**2)**0.5 * (5.1**2 + 6.2**2 + 1.1**2)**0.5)

0.5495530808754976

In [2]:
# ЗАДАНИЕ 2.2

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('netflix_titles.zip')
model = TfidfVectorizer(stop_words='english')
df['description'] = df['description'].fillna('')
feature_matrix = model.fit_transform(df['description'])

feature_matrix.shape

(7787, 17905)

In [3]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)
indices = pd.Series(df.index,index=df['title']).drop_duplicates()

def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

In [4]:
# ЗАДАНИЕ 2.3

get_recommendations('Balto')

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

In [ ]:
df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

In [ ]:
# ЗАДАНИЕ 3.1

df.apply('nunique')

# 1682

In [ ]:
# ЗАДАНИЕ 3.2

df.apply('nunique')

# 943

In [ ]:
# ЗАДАНИЕ 3.3

df['rating'].value_counts()

# 4

# в collab загрузим файл `u.data.txt`

In [ ]:
!pip install scikit-surprise
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25, random_state=13)
len(testset)

In [ ]:
# ЗАДАНИЕ 3.4

from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.25, random_state=13)
len(testset)

# 25000

In [ ]:
# ЗАДАНИЕ 3.5

from surprise import SVD, KNNBasic, accuracy
sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)
predictions

#1.

for prediction in predictions:
    if prediction.uid == '500' and prediction.iid == '699':
        print(prediction.r_ui)
        print(round(prediction.est, 2))
        break
    
# 3    
    
# 2.
uid = str(500)
iid = str(699)  
pred = knn.predict(uid, iid, verbose=True)

# 3.47

In [ ]:
# ЗАДАНИЕ 3.6

sim_options = {
    'name': 'cosine',
    'user_based': True
}
knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)
accuracy.rmse(predictions)

# 1.02

In [ ]:
# ЗАДАНИЕ 3.7

model = SVD()
model.fit(trainset)
predictions = model.test(testset)
accuracy.rmse(predictions)

# 0.94

***

## Юнит 4

https://colab.research.google.com/drive/1iZywtz2wiT8b1QvafcKURkzb4y2rEd0G

In [ ]:
!pip install lightfm
!pip install scipy==1.10

In [ ]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
import pandas as pd 

ratings = pd.read_csv('ratings.csv') # Поставленные оценки
books = pd.read_csv('books.csv') # Информация о книгах
tags = pd.read_csv('tags.csv') # Информация о тегах
book_tags = pd.read_csv('book_tags.csv') # Книги с тегами 

In [ ]:
#ЗАДАНИЕ 4.1

dict_map = dict(zip(books.goodreads_book_id,books.book_id))
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: dict_map[x])
 
book_tags[book_tags['goodreads_book_id']==5]

In [ ]:
#ЗАДАНИЕ 4.2

book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags.shape

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id))) # Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)

meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.id,book_tags.tag_id.astype("int")))) 

In [ ]:
#ЗАДАНИЕ 4.4

ratings_matrix.mean()

In [ ]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    verbose=True # Отображение обучения
)

In [ ]:
#ЗАДАНИЕ 4.5

prec_score = precision_at_k(
                     model,
                     test,
                     item_features = meta_matrix).mean() 
print(prec_score)

scores = model.predict(<индекс интересующего пользователя>, np.arange(n_items),user_features=new_user_feature)

In [2]:
import numpy as np

np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])